# TEXT MINING for PRACTICE
- 본 자료는 텍스트 마이닝을 활용한 연구 및 강의를 위한 목적으로 제작되었습니다.
- 본 자료를 강의 목적으로 활용하고자 하시는 경우 꼭 아래 메일주소로 연락주세요.
- 본 자료에 대한 허가되지 않은 배포를 금지합니다.
- 강의, 저작권, 출판, 특허, 공동저자에 관련해서는 문의 바랍니다.
- **Contact : ADMIN(admin@teanaps.com)**

---

## WEEK 04-3. 동적 페이지 수집하기: 네이버 뉴스기사
- Python을 활용해 가상의 브라우저를 띄워 웹페이지에서 데이터를 크롤링하는 방법에 대해 다룹니다.

---

> **\*\*\* 주의사항 \*\*\***  
본 자료에서 설명하는 웹크롤링하는 방법은 해당 기법에 대한 이해를 돕고자하는 교육과 이를 활용한 연구 목적으로 사용되었으며, 대량의 무단 크롤링 및 상업적 활용을 금합니다.

## 1. 데이터 수집 준비하기

### 1-1. Selenium 라이브러리 설치

In [1]:
# 가상의 브라우저를 컨트롤 할 수 있도록 도와주는 selenium 패키지를 설치합니다.
# 아래 주석을 해지하고 셀을 실행합니다.
# 설치는 한번만 수행하면 되며, 재설치시 Requirement already satisfied: ~ 라는 메시지가 출력됩니다.

#!pip install selenium

### 1-2. 라이브러리 Import 및 Chrome Driver 실행

In [2]:
# Python 코드를 통해 가상의 브라우저를 띄우기 위해 selenium 패키지를 import 합니다.
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# selenium을 활용해 브라우저를 직접 띄우는 경우, 실제 웹서핑을 할때처럼 로딩시간이 필요합니다.
# 로딩시간 동안 대기하도록 코드를 구성하기위해 time 패키지를 import 합니다.
import time

# Python 코드를 통해 웹페이지에 정보를 요청하기 위해 BeautifulSoup, urllib 패키지를 import 합니다.
from bs4 import BeautifulSoup
import requests

In [3]:
# Chrome Driver를 호출합니다.
chrome_options = webdriver.ChromeOptions()

# 브라우저에 임의로 User-agent 옵션을 넣어 Python 코드로 접속함을 숨깁니다.
chrome_options.add_argument('--user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"')

# Chrome Driver 파일의 경로를 지정하고 실행합니다.
# Chrome Driver는 아래 링크에서 다운로드 가능합니다.
# 본 Jupyter Notebook 파일과 동일한 경로에 Chrome Driver가 존재하는 경우 아래 경로를 그대로 사용합니다.

#service = Service("./chromedriver")    # Windows 운영체제
service = Service("./chromedriver")    # MAC, Linux 운영체제
                                       # 경고메시지 출력시 조치 : [시스템 환경설정] > [보안 및 개인정보 보호] > "Chrome Drive ~ 확인없이 허용"

driver = webdriver.Chrome(service=service, options=chrome_options)

## 2. 뉴스기사 및 댓글 수집하기

---

### 2-1. 뉴스기사 정보 입력

In [6]:
# 수집할 뉴스기사 정보를 입력합니다.
QUERY = "코로나19"          # 필터링 키워드
START_DATE = "2022.01.01" # 필터링 일자 (작성일 기준)
END_DATE = "2022.01.30"
START_PAGE = 1            # 검색결과 저장 페이지 범위 (네이버 뉴스기사는 검색결과 중 최대 4,000페이지만 제공)
END_PAGE = 2     
                          # 기사와 댓글을 저장할 파일명
article_filename = "article_" + QUERY + "_" + START_DATE + "_" + END_DATE + "_" + str(START_PAGE) + ".txt"
comment_filename = "comment_" + QUERY + "_" + START_DATE + "_" + END_DATE + "_" + str(START_PAGE) + ".txt"

### 2-2. 뉴스기사 수집

In [7]:
fa = open(article_filename, "w", encoding="utf-8")
fc = open(comment_filename, "w", encoding="utf-8")
news_count = 0

for page in range(START_PAGE, END_PAGE+1, 10):
    #print(page,  "번째 기시부터 최대 10개 수집중...", end="\r")
    URL = "https://search.naver.com/search.naver?&where=news&query=" + QUERY 
    URL += "&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds="
    URL += START_DATE + "&de=" + END_DATE + "&docid=&&start=" + str(page) + "&refresh_start=0"
    driver.get(URL)
    time.sleep(2)
    
    try:
        news_list = driver.find_element(By.CLASS_NAME, "list_news").find_elements(By.CLASS_NAME, "bx")
    except:
        break
    
    news_count += len(news_list)
    for news in news_list[:]:
        link_list = news.find_element(By.CLASS_NAME, "info_group").find_elements(By.TAG_NAME, "a")
        if len(link_list) == 1:
            continue
        article_url = link_list[1].get_attribute("href").strip()
        link_list[1].click()
        time.sleep(3)
        current_window = driver.current_window_handle
        try:
            new_window = [window for window in driver.window_handles if window != current_window][0]
            driver.switch_to.window(new_window)
        except:
            driver.switch_to.window(current_window)
            continue
        time.sleep(4)
        
        try:
            try:
                source_label = driver.find_element(By.CLASS_NAME, "press_logo")
            except:
                source_label = driver.find_element(By.ID, "pressLogo")
            source_img = source_label.find_element(By.TAG_NAME, "img")
            source = source_img.get_attribute("alt").strip()

            datetime = ""
            content = ""
            title = ""
            etc_good_count = ""
            etc_warm_count = ""
            etc_sad_count = ""
            etc_angry_count = ""
            etc_want_count = ""
            ent_good_count = ""
            ent_cheer_count = ""
            ent_congrats_count = ""
            ent_expect_count = ""
            ent_suprise_count = ""
            ent_sad_count = ""
            basic_good_count = ""
            basic_sad_count = ""
            basic_angry_count = ""
            basic_fan_count = ""
            basic_want_count = ""

            try:
                # 기타 섹션 기사
                datetime = driver.find_element(By.CLASS_NAME, "t11").text.strip()
                content = driver.find_element(By.CLASS_NAME, "_article_body_contents").text.strip().replace("\n", " ")
                title = driver.find_element(By.CLASS_NAME, "tts_head").text.strip()
                reaction_list = driver.find_element(By.CLASS_NAME, "end_btn").find_element(By.CLASS_NAME, "_reactionModule").find_elements(By.TAG_NAME, "a")
                etc_good_count = reaction_list[0].find_element(By.CLASS_NAME, "u_likeit_list_count").text.strip()
                etc_warm_count = reaction_list[1].find_element(By.CLASS_NAME, "u_likeit_list_count").text.strip()
                etc_sad_count = reaction_list[2].find_element(By.CLASS_NAME, "u_likeit_list_count").text.strip()
                etc_angry_count = reaction_list[3].find_element(By.CLASS_NAME, "u_likeit_list_count").text.strip()
                etc_want_count = reaction_list[4].find_element(By.CLASS_NAME, "u_likeit_list_count").text.strip()
            except:
                try:
                    # 연예 섹션 기사
                    datetime = driver.find_element(By.CLASS_NAME, "author")
                    datetime = datetime.find_element(By.TAG_NAME, "em").text.strip()
                    content = driver.find_element(By.ID, "articeBody").text.strip().replace("\n", " ")
                    title = driver.find_element(By.CLASS_NAME, "end_tit").text.strip()
                    reaction_list = driver.find_element(By.CLASS_NAME, "end_btn").find_element(By.CLASS_NAME, "_reactionModule").find_elements(By.TAG_NAME, "a")
                    ent_good_count = reaction_list[0].find_element(By.CLASS_NAME, "u_likeit_list_count").text.strip()
                    ent_cheer_count = reaction_list[1].find_element(By.CLASS_NAME, "u_likeit_list_count").text.strip()
                    ent_congrats_count = reaction_list[2].find_element(By.CLASS_NAME, "u_likeit_list_count").text.strip()
                    ent_expect_count = reaction_list[3].find_element(By.CLASS_NAME, "u_likeit_list_count").text.strip()
                    ent_suprise_count = reaction_list[4].find_element(By.CLASS_NAME, "u_likeit_list_count").text.strip()
                    ent_sad_count = reaction_list[5].find_element(By.CLASS_NAME, "u_likeit_list_count").text.strip()
                except:
                    # 일반 섹션 기사
                    datetime = driver.find_element(By.CLASS_NAME, "info").find_element(By.TAG_NAME, "span").text.replace("기사입력", "").strip()
                    content = driver.find_element(By.CLASS_NAME, "news_end").text.strip().replace("\n", " ")
                    title = driver.find_element(By.CLASS_NAME, "title").text.strip()
                    reaction_list = driver.find_element(By.CLASS_NAME, "news_end_btn").find_element(By.CLASS_NAME, "_reactionModule").find_elements(By.TAG_NAME, "a")
                    basic_good_count = reaction_list[0].find_element(By.CLASS_NAME, "u_likeit_list_count").text.strip()
                    basic_sad_count = reaction_list[1].find_element(By.CLASS_NAME, "u_likeit_list_count").text.strip()
                    basic_angry_count = reaction_list[2].find_element(By.CLASS_NAME, "u_likeit_list_count").text.strip()
                    basic_fan_count = reaction_list[3].find_element(By.CLASS_NAME, "u_likeit_list_count").text.strip()
                    basic_want_count = reaction_list[4].find_element(By.CLASS_NAME, "u_likeit_list_count").text.strip()

            review_count_list = driver.find_elements(By.CLASS_NAME, "u_cbox_count")
            if len(review_count_list) > 0:
                review_count = review_count_list[0].text.replace(",", "")
            else:
                review_count = "0"
            #print(source + "\t" + datetime + "\t" + review_count + "\t" + 
            #      good_count + "\t" + warm_count + "\t" + sad_count + "\t" + 
            #      angry_count + "\t" + want_count + "\t" + article_url + "\t" + 
            #      title + "\t" + content)
            print(" "*100, end="\r")
            print(str(page)+"/"+str(END_PAGE+1)+" Page,", title + " (댓글: "+str(review_count)+"건)", end="\r")
            fa.write(source + "\t" + datetime + "\t" + review_count + "\t" + 
                     ent_good_count + "\t" + ent_cheer_count + "\t" + ent_congrats_count + "\t" + 
                     ent_expect_count + "\t" + ent_suprise_count + "\t" + ent_sad_count + "\t" + 
                     basic_good_count + "\t" + basic_sad_count + "\t" + basic_angry_count + "\t" + 
                     basic_fan_count + "\t" + basic_want_count + "\t" + 
                     etc_good_count + "\t" + etc_warm_count + "\t" + etc_sad_count + "\t" + 
                     etc_angry_count + "\t" + etc_want_count + "\t" + 
                     article_url + "\t" + 
                     title + "\t" + content + "\n")
            # 댓글 더보기 클릭
            if int(review_count) > 0:
                driver.find_element(By.CLASS_NAME, "u_cbox_in_view_comment").click()
                time.sleep(3)

                # 더보기 버튼 클릭
                for i in range(10):
                    try:
                        more_button_status = driver.find_element(By.CLASS_NAME, "u_cbox_paginate").get_attribute("style").strip()
                        if more_button_status == '':
                            driver.find_element(By.CLASS_NAME, "u_cbox_more_wrap").click()
                        time.sleep(1)
                    except:
                        continue

                # 답글 클릭
                comment_list = driver.find_elements(By.CLASS_NAME, "u_cbox_area")
                driver.execute_script('window.scrollTo(0, 0);')
                for comment in comment_list:
                    reply_count_list = comment.find_elements(By.CLASS_NAME, "u_cbox_btn_reply")
                    if len(reply_count_list) > 0:
                        reply_count = reply_count_list[0].text.strip()
                        if reply_count != "답글0":
                            #reply_button = comment.find_element(By.CLASS_NAME, "u_cbox_btn_reply")
                            #reply_button.click()
                            reply_count_list[0].click()
                time.sleep(3)

                # 댓글 수집
                driver.execute_script('window.scrollTo(0, 0);')
                comment_box_list = driver.find_elements(By.CLASS_NAME, "u_cbox_area")
                for i in range(len(comment_box_list)):
                    comment_box = comment_box_list[i]
                    comment_nick = comment_box.find_element(By.CLASS_NAME, "u_cbox_nick").text
                    comment_content = comment_box.find_element(By.CLASS_NAME, "u_cbox_text_wrap").text
                    comment_datetime = comment_box.find_element(By.CLASS_NAME, "u_cbox_date").text
                    try:
                        comment_good = comment_box.find_element(By.CLASS_NAME, "u_cbox_cnt_recomm").text
                        comment_bed = comment_box.find_element(By.CLASS_NAME, "u_cbox_cnt_unrecomm").text
                    except:
                        comment_good = "0"
                        comment_bed = "0"
                    reply_count_list = comment_box.find_elements(By.CLASS_NAME, "u_cbox_reply_cnt")
                    if len(reply_count_list) > 0:
                        is_reply = "1"
                        parent_id = "-1"
                        temp_parent_id = i
                        reply_count = reply_count_list[0].text
                    else:
                        is_reply = "0"
                        parent_id = str(temp_parent_id)
                        reply_count = "0"
                    #print(str(i) + "\t" + is_reply + "\t" + reply_count + "\t" + 
                    #      parent_id + "\t" + comment_nick + "\t" + comment_datetime + "\t" + 
                    #      comment_good + "\t" + comment_bed + "\t" + comment_content + "\t" +
                    #      article_url)
                    fc.write(str(i) + "\t" + is_reply + "\t" + reply_count + "\t" + 
                             parent_id + "\t" + comment_nick + "\t" + comment_datetime + "\t" + 
                             comment_good + "\t" + comment_bed + "\t" + comment_content + "\t" + 
                             article_url + "\n")
        except:
            driver.close()
            time.sleep(3)
            driver.switch_to.window(current_window)
            fa.flush()
            fc.flush()
            continue
            
        driver.close()
        time.sleep(3)
        driver.switch_to.window(current_window)
        fa.flush()
        fc.flush()
        
fa.close()
fc.close()

# 수집종료
print()
print("* 최대", news_count, "개 기사 수집이 완료되었습니다.")
print("* 수집된 기사는 아래 파일에 저장되었습니다.")
print("  - 기사본문 :", article_filename)
print("  - 댓글 :", comment_filename)

1/3 Page, [신년사]이재명 "오직 국민, 오직 민생 각오로 경제회복에 온 힘" (댓글: 18건)                                           
* 최대 10 개 기사 수집이 완료되었습니다.
* 수집된 기사는 아래 파일에 저장되었습니다.
  - 기사본문 : article_코로나19_2022.01.01_2022.01.30_1.txt
  - 댓글 : comment_코로나19_2022.01.01_2022.01.30_1.txt


In [8]:
driver.close()

> **\*\*\* TIP \*\*\***  
새탭에서 기사가 열린 후 수집이 종료된 경우 아래 셀을 실행합니다.

In [ ]:
driver.switch_to.window(current_window)